In [1]:
from agents import Agent, Runner, TResponseInputItem, RunContextWrapper, handoff
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
simple_agent = Agent(
    name = "Bob",
    instructions= "You are a friendly assistant. Reply concisely."
)

result = await Runner.run(simple_agent, "Hie")
result.final_output

'Hello! How can I help you today? 😊'

In [ ]:
convo: list[TResponseInputItem] = []
print("You are now chatting with agent Bob. Type 'exit' to end the conversation.")

while True:
    user_input = input("You: ") 
    print("You: " + user_input)
    if user_input.lower() == "exit":
        print("Ending the conversation. Goodbye!")
        break
    convo.append({"content":user_input, "role":"user"})
    result = await Runner.run(simple_agent, convo)

    print(f"Agent: {result.final_output}")

    convo = result.to_input_list()

You are now chatting with agent Bob. Type 'exit' to end the conversation.
You: Hie
Agent: Hello! How can I assist you today?
You: Are you Bob?
Agent: No, I’m not Bob—I’m your AI assistant! How can I help you today?
You: How can I call you/
Agent: You can call me whatever you like! Some people use names like "Assistant," "AI," or even give me a unique name. Do you have a name in mind for me?
You: yes
Agent: Great! What name would you like to call me?
You: Bob
Agent: Perfect! You can call me Bob. How can I help you today?
You: What are you specialities
Agent: As Bob, your friendly assistant, I’m here to help with a wide range of tasks! My specialties include:

- Answering questions and explaining concepts
- Helping with research and finding information
- Providing study tips and summaries
- Assisting with writing, editing, and proofreading
- Offering recommendations (books, movies, travel, etc.)
- Helping with math and problem-solving
- Giving tech support and troubleshooting advice
- Of

### Example

In [7]:
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

def on_math_handoff(context: RunContextWrapper[None]):
    print("Handoff to Math Tutor Agent")

def on_history_handoff(context: RunContextWrapper[None]):
    print("Handoff to History Tutor Agent")

triage_agent = Agent(
    name="Triage Agent",
    instructions="""
You are a routing agent ONLY.

Rules:
- If the question is about math (numbers, arithmetic, algebra, division, etc),
  you MUST hand off to the Math Tutor agent.
- If the question is about history (past events, dates, civilizations, wars),
  you MUST hand off to the History Tutor agent.
- DO NOT answer math or history questions yourself.
- ONLY hand off.
""",
    handoffs=[handoff(history_tutor_agent, on_handoff=on_history_handoff), 
              handoff(math_tutor_agent, on_handoff=on_math_handoff)]
)

convo: list[TResponseInputItem] = []
last_agent: Agent = triage_agent
print("You are now chatting with agent Bob. Type 'exit' to end the conversation.")

while True:
    user_input = input("You: ") 
    print("You: " + user_input)

    if user_input.lower() == "exit":
        print("Ending the conversation. Goodbye!")
        break
    
    convo.append({"content":user_input, "role":"user"})
    result = await Runner.run(triage_agent, convo)

    convo = result.to_input_list()
    last_agent = result.last_agent
    # print(result.last_agent)
    print(f"{last_agent.name}:{result.final_output}\n")

You are now chatting with agent Bob. Type 'exit' to end the conversation.
You: tell me about sindh
Handoff to History Tutor Agent
History Tutor:Sindh is a province in southeastern Pakistan that has a rich and ancient history. It is located along the lower Indus River and borders the Arabian Sea to the south. Here are some important points about Sindh:

1. Ancient Civilization: Sindh is home to some of the oldest sites of the Indus Valley Civilization (also known as the Harappan Civilization), which flourished around 2500 BCE. Notable archaeological sites include Mohenjo-daro, one of the world's earliest major cities.

2. Strategic Location: Historically, Sindh has been a gateway between South Asia and the Middle East, as it is situated along important trade and migration routes.

3. Islamic Conquest: Sindh was one of the first parts of the Indian subcontinent to come under Islamic rule. In 711 CE, the Arab general Muhammad bin Qasim conquered Sindh, making it the easternmost province o